<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

Orchestrator-Worker Workflow Design Pattern

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)

In [ ]:

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [ ]:
request = "Please come up with a challenging, nuanced problem in the field of farming that I can ask a LLM."
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [ ]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "openai/gpt-oss-120b"

response = groq.chat.completions.create(model=model_name, messages=messages)
question = response.choices[0].message.content

display(Markdown(question))

In [ ]:
the_workers = ["gpt-5-nano", "claude-sonnet-4-5", "gemini-2.5-flash", "openai/gpt-oss-120b", "llama3.2"]
the_orchestrator = "openai/gpt-oss-120b"

def gpt_nano_use(messages, model_name):
    openai = OpenAI()
    response = openai.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content
    return answer

def claude_use(messages, model_name):
    claude = Anthropic()
    response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
    answer = response.content[0].text
    return answer

def gemini_use(messages, model_name):
    gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
    response = gemini.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content
    return answer

def gpt_oss_use(messages, model_name):
    groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
    response = groq.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content
    return answer

def llama_use(messages, model_name):
    ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    response = ollama.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content
    return answer


In [ ]:
task = f"Please break down the following question into sub-problems: {question}."
task += f"Then select model that you think will provide the best response quality per sub-problem out of {the_workers}, you are allowed to repeat models if you think that is necessary. "
task += f"Answer only with the sub-problems and the models that you have selected, no explanation."
task += f"Provide answer in JSON format: {{'sub-problem': 'model', 'sub-problem': 'model', ...}}"
messages = [{"role": "user", "content": task}]

groq_orchestrator = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
response = groq_orchestrator.chat.completions.create(model="openai/gpt-oss-120b", messages=messages)
answer = response.choices[0].message.content
display(Markdown(answer))


In [ ]:
backbone_messages = []
answer_combination = []
for sub_problem, model in json.loads(answer).items():
    backbone_messages.append(f"Sub-problem: {sub_problem}, Model: {model}")
    if model == "gpt-5-nano":
        answer_combination.append(gpt_nano_use(messages, model))
    elif model == "claude-sonnet-4-5":
        answer_combination.append(claude_use(messages, model))
    elif model == "gemini-2.5-flash":
        answer_combination.append(gemini_use(messages, model))
    elif model == "openai/gpt-oss-120b":
        answer_combination.append(gpt_oss_use(messages, model))
    elif model == "llama3.2":
        answer_combination.append(llama_use(messages, model))

print(answer_combination)

In [ ]:
synthesis_messages = [{"role": "user", "content": f"synthesize all of these {answer_combination} and provide the final response to the {question}"}]
response = groq_orchestrator.chat.completions.create(model="openai/gpt-oss-120b", messages=synthesis_messages)
final_answer = response.choices[0].message.content
display(Markdown(final_answer))


Cross Verify with a Solo Model

In [ ]:
messages = [{"role": "user", "content": f"provide a response to answer {question}"}]
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
response = groq.chat.completions.create(model="openai/gpt-oss-120b", messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))

Key Takeaways

1. The amount of raw detail and facts (statistics, data, etc.) that is provided from the workflow incorporating many models is far greater than a single model.
2. The solo model is still was more coherent because all the models in the workflow were proposing different strategies, so direction of the initiatize was diluated. 